# Autograd and Optimisation

Almost every scientific area, especially experimental work requires function fitting, for example to find the parameters for a theory that describe the data best. One classical example is linear regression, which we discussed in the Assignment 6 on day 2. Here we assume that two sets of data x, and y are related by a linear function, $f(x)=w^Tx$. Of course most datasets are much more complex. While for many standard models there are already packages that automate packages (for example sklearn for standard machine learning models, which we discuss later), you are often on your own if your goal is to adapt or change a model. 

We will now talk about model fitting in a more general context: most fitting approaches find a set of parameters that minimize some measure of error, for example the expected squared difference between model prediction and real data, mean squared error. We will therefore start first with the question: how can we find a (local) minimum of an arbitrary function $f$.

Scipy is a scientific package that includes tools for a lot of scientific numeric purposes, including a package for optimisation. 

In [1]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


We will use the subpackage scipy.optimize 

https://docs.scipy.org/doc/scipy/reference/optimize.html

In [2]:
import scipy.optimize as optim

## Functions with several return values: defining an optimisation problem

We want to find a minimum of a simple function $f(x)=1/2 x^Tx$

where $x$ is a n dimensional vector. Efficient optimisation requires the knowledge of the gradient of the function, i.e., the partial derivatives of $f$ wrt each parameter. for our function $f$ the gradient is $g(x)=x$. For scipy.minimize, we need to return both the result f and the gradient g from the same function call, we will introduce this in the following:

In [3]:
import numpy as np
def problem(x):
    #compute the value f:
    f=0.5 * np.inner(x,x) #np.inner(x,y) ciomputes x^Ty you could also implement this as np.sum(x**2)
    #compute the gradient g
    g = x 

    #returning several values returns can be done with a simple comma separated list. This makes the function return a tuple (f,g)
    return f,g

In [4]:
x = np.ones(10)
result = problem(x)
result

(5.0, array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [5]:
#python allows a short cut: we can assign a tuple to a comma separated list of exactly the same size:
f,g = result
print(f)
print(g)

5.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [6]:
#and we can do this in one line. we saw this already on day 2 with the function np.linalg.eigh which returned eigenvalues and eigenvectors

f,g = problem(x)
print(f)
print(g)

5.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


We now want to minimize the function using scipy.optimize.minimize.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize

We make use of another cool feature of python: functions can take another function as argument.  We use that to tell minimize that it should try to find the minimum of our defined function "problem". The only thing we need to tell it is an initial guess for a solution: the starting point of the optimisation as well as tell it that our problem returns a gradient (called jacobian or jac in the minimisation package)

In [7]:
x_init=np.array([1,2,3,4])
optimResult = optim.minimize(problem,x_init, jac=True)

The result object is a class object which stores mostly some interesting data about the final result of the optimisation

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html#scipy.optimize.OptimizeResult

We can print out the result object and it provides some (more or less) valueable information:
- success: Was the optimisation successful?
- message: a human readable message for the error status
- status: a computer interpretable sgtatus code for the message
- x: the final guess for the minimum
- fun: the final achieved function value, the value of at problem(x)
- nit/nfev: the number of iterations/function evaluations the algorithm took
- jac: the value of g returned at problem(x)

In [8]:
optimResult

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 8.16594296420188e-32
        x: [-5.551e-17 -3.331e-16 -2.220e-16  0.000e+00]
      nit: 3
      jac: [-5.551e-17 -3.331e-16 -2.220e-16  0.000e+00]
 hess_inv: [[ 1.000e+00 -1.388e-17  1.388e-17  5.551e-17]
            [-1.388e-17  1.000e+00  2.776e-17  5.551e-17]
            [ 1.388e-17  8.327e-17  1.000e+00  5.551e-17]
            [ 2.776e-17  0.000e+00  1.110e-16  1.000e+00]]
     nfev: 4
     njev: 4

In [9]:
xOpt = optimResult.x

problem(xOpt)

(8.16594296420188e-32,
 array([-5.55111512e-17, -3.33066907e-16, -2.22044605e-16,  0.00000000e+00]))

# Autodifferentiation

Finding the gradient of a function is error prone. For this reason, and due to the need of automating this for the big models in machine learning, there are a large number of libraries that implement automatic differentiation. The most well known packages are pytorch, tensorflow and jax. But those libraries are so large and complex(for example they also offer full gpu support) that we cannot introduce them here. They would be part of a proper machine learning course.

instead we use a smaller package called autograd, which is just "numpy with gradients". 

https://github.com/HIPS/autograd

#the next lines installs autograd with support for scipy

In [10]:
%pip install autograd[scipy]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Before you continue, please restart the kernel! We do not want normal numpy to interfere with what we do next.

Autograd comes with its own version of numpy and a small set of tools that interact with this version of numpy. It supports almost everything that normal numpy supports with only a few exceptions. e..g, if you have a function with argument x and you want to compute the derivative wrt x, then you cannot use indexing to assign new values to x inside the function. But otherwise everything works, for example even +=,-= etc.

We use this to implement linear regression as in assignment 6.2 on day 2. And we use the mean squared error function from task 7.2

In [11]:
import autograd.numpy as np # please see that we are using the autograd numpy package now!

First we implement our model class $model(x,w)$. We interpret this as: the prediction of a model with parameters w given a new data input x.
For linear regression:  $model(x,w)=w^Tx$. 

We implement it to support batching, i.e., computing the result for not a single vector x but a matrix X where each row is a data point.
In this case: $model(X,w)=Xw$. We call this a linear model

In [12]:
def linear_model(X,w):
    return X@w

Now we implement the mean squared error $mse=mean[(model(x,w)-y)^2]$. 
To make this reusable, we will take $f$ as function argument, like in minimize before!
Parameters of mse:
- X the matrix of points
- y a vector of ground truth labels (our measurement data)
- model the model class we want to use for the prediction
- w the parameters of f.

In [13]:
def mse(X,y,model,w):
    prediction = model(X,w) #our model prediction for each point in X
    errors = (prediction - y)**2 #(f(x,w)-y)^2 for all our data points
    return np.mean(errors) #return the average of all errors. note that here np is no longer numpy.mean but autograd.numpy.mean!
    

We now generate a problem using the same code as in assignment 6.2 day 2.

In [14]:
# A little help for X.
# We want to sample a simple 1D problem with N=200 points.
# we add a second column that is constant 1,which allows us to learn a constant offset parameter. 
# Thus we actually compute f(x)=w[0]x+w[1]
N=200
X = 10*np.random.randn(N,2) #N points with 2 values each
X[:,-1] = 1 #overwrite the last column (using -1 here instead of "1" allows us to change the dimensionality of the data!)
w_truth = np.array([2.0,3.0]) #the true unknown values
#Subtask 1: compute y
# for this first generate a normal distributed vector epsilon with N entries
# then compute y_i= w^Tx_i+epsilon_i
epsilon= np.random.randn(N)
y = linear_model(X,w_truth)+epsilon
print(y.shape)

(200,)


Let us start computing the mean squared error using the values of w_truth used to generate the data

In [15]:
mse(X,y,linear_model, w_truth)

0.9968248895442396

Now we want to minimize. For this, we need to compute our gradient. Introduce the key tool of autograd:
The function autograd.value_and_grad takes a function f(x) and creates a new problem that returns a tuple: f(x) and g(x)

In [16]:
from autograd import value_and_grad

#for our optimiosation problem, we need to create a function that only depends on the parameters w.
def problem_f(w):
    #note that the variables X,y, and the function linear_model that we defined above are also accessible INSIDE f!
    #so we can just call mse with the first three arguments as before!
    return mse(X,y,linear_model, w) 

#the problem we want to minimize!
problem = value_and_grad(problem_f)

#let us try it out by computing value and gradient at the true value
f,g = problem(w_truth)
print(f)
print(g)

0.9968248895442396
[0.83225316 0.13791519]


In [17]:
from scipy.optimize import minimize
w_init = np.ones(2)
optimResult = minimize(problem, w_init,jac=True)


In [18]:
optimResult

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.9904571669423347
        x: [ 1.996e+00  2.933e+00]
      nit: 4
      jac: [ 1.130e-14  6.054e-16]
 hess_inv: [[ 5.301e-03 -2.008e-03]
            [-2.008e-03  5.008e-01]]
     nfev: 6
     njev: 6

# Challenge Autodiff, and Classes (harder to HARD)

We would like to have our code easier to understand and more modular. For example, right now mse needs to take 4 arguments, X,y,model and w. 
We can use classes to make this possible and order our code so that mse does not even need to know that f has arguments!

Check the tutorial on classes and read the python docs on classes as needed. We want to implement a new set of classes where each model follows a specific interface. 

- \_\_init\_\_: initializes the parameters of the model so that can it be used. This means often that init needs to take a few parameters which say how big internal arrays should be, for example what the dimensionality of the data is we want to use the model for.
- w=model.getParameters(): a function returning a vector that appends all parameters of the model to a single array
- model.setParameters(w): a function that sets all parameters of the model. The idea is that if we have two models a nd use getParameters on the one model and then use setParameters(w) on the other model, they will afterwise produce the same predictions.
- model.predict(X): Returns the predictions of the model for input X using the currently set parameters. you can also instead rename the function ton  \_\_call_\_\_ in which case you can write model(X) insteads of model.predict(X).


Implement a new class LinearModel that follows that interface. It is supposed to have one data attribute
x.w which is a numpy array that holds the linear parameters. This means:

- \_\_init\_\_: Takes one parameter n and adds model.w as a random normal distributed vector of length n.
- model.getParameters(): returns model.w
- model.setParameters(w): sets model.w to the new value.
- model.predict(X): X@w

Implement a new function mse which takes a model and returns the mean squared error.
Then implement a new problem_f method which uses first model.setParameters to set the model to the new parameter values that are provided as argument and then computes mse for the model and data. Then fit the model parameters using minimize. As starting point you can use the value returned by model.getParameters().


(HARD) You can then challenge yourself by implementing a more complex application, for example a neural network with a hidden layer with K neurons:

$f(x)=b_2+w^T_2 h(W_1x+b_1)$

here, h is a elementwise nonlinear function, like h(x)=np.maximum(x,0) (the ReLU activation function in machine learning). $W_1$ is a matrix with K rows and number of columns the same as the dimensions of the inoput points x. $w_2,b_1$ are vectors of length $K$ and $b_2$ is a scalar. These 4 objects make up the parameters of the neural network. To implement setParameters you can use slicing, e.g., model.w_2=w\[0:K\]  to set w_2 to the first K values in w. you should also look up np.reshape to transform a vector into a matrix with the same number of elements (e.g., to turn the sliced parameters in w into the matrix W_1. For getParameters() you can use W_2.flatten() to turn a matrix into a vector and use np.concatenate to create a vector w from the 4 components.

then get ayour favourite dataset (or make something up) and try whether you can get it to work!
